In [1]:
import modulos as md
import numpy as np
import pandas as pd
import pandas_ta as pta

'''
symbol='SUIUSDT'
lista=md.lista_de_monedas ()

for symbol in lista:
    try:
        data = md.obtiene_historial(symbol)
        data = md.estrategia_bb(data)
        resultado = md.backtesting(data)
        if float(resultado[4]) > 1015:
            print(f"{symbol} - {resultado[4]}")
    except:
        pass        
'''


symbol = "MATICUSDT"
marketTF = "1d"
rsiMom = 70
useRsi = False
res = "1w"

def f_sec(df, timeframe):
    # convertir la columna timestamp a un objeto datetime
    df['Open Time'] = pd.to_datetime(df['Open Time'])
    # calcular la diferencia de tiempo entre filas adyacentes
    time_diff = df['Open Time'].diff()
    # determinar si la barra actual ha terminado
    if time_diff.iloc[-1] == pd.Timedelta(timeframe):
        # la barra actual ha terminado
        return 0
    else:
        # la barra actual aún no ha terminado
        return 1

# Get daily data
df = md.obtiene_historial(symbol, marketTF)
# Get weekly data
df_weekly = md.obtiene_historial(symbol, res)
# Calculate EMA for weekly data
ema_weekly = pta.ema(df_weekly['Close'],20)
if f_sec(df, res) == 1:
    ema_weekly.iloc[-1] = ema_weekly.iloc[-2]
# Resample EMA to daily data
ema_weekly = ema_weekly.reindex(df.index, method='nearest')
df['emaweek']=ema_weekly
df['emaweek']=df['emaweek'].shift(9)
df['color_emaweek']=np.where(df.Close>df.emaweek, 'green', 'red')
# Get ATR value
df['atrValue'] = pta.atr(df['High'], df['Low'], df['Close'], length=5)
# Check if price is above or beLow EMA filter
df['regimeFilter'] = df.Close > (df.emaweek + (df.atrValue * 0.25))
# Calculate RSI
df['rsiValue'] = pta.rsi(df['Close'], length=7)
# Get bullish momentum filter
df['bullish'] = df['regimeFilter'] & (df.rsiValue.any() > rsiMom or not useRsi)
# Define bearish caution filter
df['caution'] = df['bullish'] & ((df['High'].rolling(window=7).max() - df['Low']) > (df['atrValue'] * 1.5))
# Set momentum color
df['bgCol'] = 'red'
df['bgCol'] = np.where(df['bullish'].shift(-1),'green',df['bgCol'])
df['bgCol'] = np.where(df['caution'].shift(-1),'orange',df['bgCol'])

mult_take_profit = 1
mult_stop_loss = 1
df['signal'] = np.where(
    df.bullish & ~ df.caution
    ,1,
    0
    )
 
df['take_profit'] = df.Close + (df.atrValue * mult_take_profit)
df['stop_loss'] = df.Close - (df.atrValue * mult_stop_loss)


resultado = md.backtesting(df)
resultado


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Start                     2020-10-01 00:00:00
End                       2023-06-27 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        29.8
Equity Final [$]                  1697.237011
Equity Peak [$]                   2562.075424
Return [%]                          69.723701
Buy & Hold Return [%]             3416.332982
Return (Ann.) [%]                   21.298662
Volatility (Ann.) [%]                49.44904
Sharpe Ratio                         0.430719
Sortino Ratio                        0.821617
Calmar Ratio                         0.469066
Max. Drawdown [%]                  -45.406554
Avg. Drawdown [%]                   -9.928126
Max. Drawdown Duration      571 days 00:00:00
Avg. Drawdown Duration       68 days 00:00:00
# Trades                                   82
Win Rate [%]                        56.097561
Best Trade [%]                      26.697351
Worst Trade [%]                    -30.678656
Avg. Trade [%]                    